In [5]:
%%writefile data1.csv # magic 명령어인 %%은 매직이라 맨 처음에 써야함 ㅎㅎ
time,temperature,humidity
0,22,58
1,21,57
2,25,57
3,26,55
4,22,53

Writing data1.csv


In [6]:
import dask.dataframe as dd #Dask 패키지의 dataframe 서브패키지 임포트

In [9]:
df = dd.read_csv("data1.csv")
df #df는 데이터프레임과 유사하지만 실제로 데이터가 메모리에 없어
#값은 표시되지 않는다

,time,temperature,humidity
npartitions=1,,,
,int64,int64,int64
,...,...,...


In [11]:
df.tail()

,time,temperature,humidity
0,0,22,58
1,1,21,57
2,2,25,57
3,3,26,55
4,4,22,53


In [12]:
df.temperature.mean()
#결과 바로 안나옴. 왜냐하면 연산 반환값이 작업task기 때문
#구체적으로 어떤 작업인지 보려면 visualize method 사용해야
#실제로 구하려면 결과로 받은 작업 객체의 compute method 호출

dd.Scalar<series-..., dtype=float64>

In [13]:
df.temperature.mean().compute()

23.2

In [14]:
(df.temperature * 9 / 5 + 32).compute()

0    71.6
1    69.8
2    77.0
3    78.8
4    71.6
Name: temperature, dtype: float64

In [15]:
#temperature 열 갱신 : pandas 문법 못 쓰고 assign 메서드 사용
#assign할 때는 compute할 필요 없음
df = df.assign(temperature = df.temperature * 9 / 5 + 32)
df.head()

,time,temperature,humidity
0,0,71.6,58
1,1,69.8,57
2,2,77.0,57
3,3,78.8,55
4,4,71.6,53


In [19]:
#자료형 변환 또는 새로운 열 추가 가능
df = df.assign(title = df.temperature.astype('str') + "degree")
df.head()

,time,temperature,humidity,title
0,0,71.6,58,71.6degree
1,1,69.8,57,69.8degree
2,2,77.0,57,77.0degree
3,3,78.8,55,78.8degree
4,4,71.6,53,71.6degree


In [20]:
#복수 데이터에 대한 가상 데이터 프레임
#가상 데이터프레임으로 원천source데이터 파일을 복수 설정 가능

In [21]:
%%writefile data2.csv
time,temperature,humidity
0,22,58
1,21,57
2,25,57
3,26,55
4,22,53
5,23,59

Writing data2.csv


In [22]:
%%writefile data3.csv
time,temperature,humidity
0,22,58
1,21,57
2,25,57
3,26,55
4,22,53
5,23,59

Writing data3.csv


In [23]:
#복수 파일은 와일드카드(*)기호 이용
df = dd.read_csv('data*.csv')
df.count().compute()

time           17
temperature    17
humidity       17
dtype: int64

In [24]:
df.temperature.describe().compute()

count    17.000000
mean     23.176471
std       1.878673
min      21.000000
25%      22.000000
50%      22.500000
75%      25.000000
max      26.000000
Name: temperature, dtype: float64

In [28]:
!wget -O crime.csv https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2020-07-14 12:53:27--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.68.26, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: 'crime.csv'

     0K .......... .......... .......... .......... ..........  259K
    50K .......... .......... .......... .......... ..........  213K
   100K .......... .......... .......... .......... .......... 5.15M
   150K .......... .......... .......... .......... ..........  102K
   200K .......... .......... .......... .......... .......... 7.08M
   250K .......... .......... .......... .......... .......... 10.2M
   300K .......... .......... .......... .......... ..........  266K
   350K .......... .......... .......... .......... ..........  272K
   400K .......... .......... .........

In [34]:
df = dd.read_csv("crime.csv", dtype=str, error_bad_lines=False, warn_bad_lines=False)
df

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
npartitions=5,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [35]:
df.tail()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
57191,11269051,JB202570,03/27/2018 08:00:00 PM,058XX N GLENWOOD AVE,0560,ASSAULT,SIMPLE,APARTMENT,false,false,...,48,77,08A,1165931,1938685,2018,05/04/2018 03:51:04 PM,41.987338819,-87.665062403,"(41.987338819, -87.665062403)"
57192,11269052,JB202572,03/23/2018 11:00:00 AM,020XX W 52ND PL,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,OTHER,false,false,...,16,61,07,1163831,1869680,2018,05/04/2018 03:51:04 PM,41.798028449,-87.674734399,"(41.798028449, -87.674734399)"
57193,"""message"" : ""Internal error""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57194,"""status"" : 500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57195,},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
#dask는 작업진행도를 알 수 있는 ProgressBar 제공
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

In [37]:
%%time
df.count().compute()

[########################################] | 100% Completed | 13.6s
Wall time: 13.6 s


ID                      1150066
Case Number             1150062
Date                    1150063
Block                   1150063
IUCR                    1150063
Primary Type            1150063
Description             1150063
Location Description    1144720
Arrest                  1150063
Domestic                1150063
Beat                    1150063
District                1150062
Ward                    1140041
Community Area          1140120
FBI Code                1150063
X Coordinate            1081182
Y Coordinate            1081182
Year                    1150063
Updated On              1150063
Latitude                1081182
Longitude               1081182
Location                1081182
dtype: int64

In [39]:
%%time
df.count().compute(scheduler = 'processes', num_workers = 4)
#멀티프로세싱 & 4개의 CPU 코어 동시 사용
#dask는 대량 데이터의 분석 작업을 돕기 위해 작업 스케줄러task scheduler 제공
#task scheduler : 하나의 작업을 여러 개의 thread, process, node 나누어 분담

[########################################] | 100% Completed |  8.8s
Wall time: 8.88 s


ID                      1150066
Case Number             1150062
Date                    1150063
Block                   1150063
IUCR                    1150063
Primary Type            1150063
Description             1150063
Location Description    1144720
Arrest                  1150063
Domestic                1150063
Beat                    1150063
District                1150062
Ward                    1140041
Community Area          1140120
FBI Code                1150063
X Coordinate            1081182
Y Coordinate            1081182
Year                    1150063
Updated On              1150063
Latitude                1081182
Longitude               1081182
Location                1081182
dtype: int64

In [40]:
!rm data?.csv

'rm'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
